<a href="https://colab.research.google.com/github/angulorojasmariaclaudia-coder/TRABAJO-FINAL-TECNICAS-DE-CIENCIAS-DE-DATOS/blob/main/Configuraci%C3%B3n_de_C%C3%B3digo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **¡¡IMPORTANTE!! Para enterder el DATASET**

🎯 **OBJETIVO REAL**

Clasificar qué tan mal contaminada está cada Comunidad Autónoma (CCAA)
usando solo los contaminantes, no enfermedades.

\

🎯 Para clasificar la contaminación de una CCAA, necesitamos **solo los contaminantes**:
* Air Pollution Average [ug/m3]
* Air Pollution Population Weighted Average[ug/m3]
* Air Pollutant (pivotado a PM25/NO2/O3)
* Year
* Affected Population
* Populated Area [km2]
* CCAA: Comunidad Autónoma

\

🧠 El conjunto de datos contiene registros “**Mortality**” con las enfermedades como 'Diabetes' o cánceres. **NO** debemos usar `Mortality`, `Outcome`, `Attributable deaths (AD)`, `Value` (valor estimado de muertes en base a la OMS), ni `Value for 100k…`. Porque es información derivada de la contaminación.

La parte de “enfermedades” solo sirve para:

* ✔️ tu análisis exploratorio
* ✔️ entender impacto de salud
* ❌ NO para el modelo


\

**ELIMINAREMOS LAS ENFERMEDADES DEL DATASET**

\

**IMPORTANTE TAMBIÉN:**
* Años considerados: 2018 a 2022.
* Edad de la población >= 25 (población expuesta a la contaminación).
* Contaminantes: NO2, PM2.5	y O3.
* Población: Urban Centres (grid).


# Paquetes

In [ ]:
import pandas as pd
from google.colab import files

# Conjunto de datos

In [ ]:
uploaded = files.upload()
df = pd.read_csv('causas3.csv')

Saving causas3.csv to causas3 (9).csv


In [ ]:
df.shape # numero de filas y columnas

(25440, 23)

In [ ]:
df['Year'].value_counts() # años de estudio

,count
Year,
2022,11040
2021,10080
2019,1440
2020,1440
2018,1440


In [ ]:
df.head(2) # dataset original

,City Boundary Specification (LAU/grid),Country Or Territory,City Or Territory,City Code,Year,Air Pollutant,Data Aggregation Id,Scenario,Category,Outcome,...,Affected Population,Populated Area [km2],Air Pollution Average [ug/m3],Air Pollution Population Weighted Average [ug/m3],Value,Value - lower CI,Value - upper CI,Value for 100k Of Affected Population,Value for 100k Of Affected Population - lower CI,Value for 100k Of Affected Population - upper CI
0,Urban Centres (grid),Spain,Collado Villalba,GEOSTAT11_748,2021,NO2,P1Y,Baseline from WHO 2021 AQG,Mortality,Diabetes Mellitus,...,41314,9.0,13.0,13.8,8,4,11,19,10,28
1,Urban Centres (grid),Spain,Collado Villalba,GEOSTAT11_748,2021,O3,SOMO35,Baseline from WHO 2021 AQG,Mortality,All causes,...,54686,9.0,4742.1,4735.3,2,2,3,5,4,6


# Preparación del conjunto de datos

### Añadir variables CCAA

Vamos a ver las ciudades que reúne nuestro conjunto de datos.

In [ ]:
df["City Or Territory"].unique()

array(['Collado Villalba', 'Terrassa', 'Toledo', 'Santiago de Compostela',
       'Sevilla', 'Santa Monica', 'Santander', 'Córdoba', 'Tomares',
       'Talavera de la Reina', 'Torrejón de Ardoz', 'Coslada',
       'Tarragona', 'Torrelavega', 'Cuenca', 'Dos Hermanas',
       'Torremolinos', 'Eivissa', 'Torrevieja', 'el Masnou',
       "El Port de Sagunt / Canet d'En Berenguer", 'Valdemoro',
       'El Prat de Llobregat', 'Valencia', 'Valladolid', 'Elche/Elx',
       'Vigo', 'Elda', 'Viladecans', 'Ferrol', 'Vilanova i la Geltrú',
       'Fuengirola', 'Vitoria/Gasteiz', 'Fuenlabrada', 'Gandia', 'Zamora',
       'Gijón', 'Zaragoza', 'Girona', 'Granada', 'Granollers',
       'Guadalajara', 'Huelva', 'Igualada', 'Irun', 'Jaén',
       'Jerez de la Frontera', 'León', 'Linares',
       'Línea de la Concepción, La', 'Lleida', 'Logroño', 'Lorca', 'Lugo',
       'Madrid', 'Majadahonda', 'Málaga', 'Manresa', 'Marbella', 'Mataró',
       'Melilla', 'Mérida', 'A Coruña', 'Albacete', 'Mollet del Vall

Vemos que hay dos casillas, Santa Mónica y All Urban Centres in a Country, que no son ciudades de España. Por lo que las quitaremos de nuestro conjunto de datos.

In [ ]:
df = df[df["City Or Territory"] != "All Urban Centres in a Country"]
df = df[df["City Or Territory"] != "Santa Monica"]

A continuación, vamos a añadir una columna que indique la Comunidad Autónoma según la ciudad registrada en nuestro conjunto de datos.

In [ ]:
mapa_ccaa = {
    'Collado Villalba': 'Comunidad de Madrid', 'Terrassa': 'Cataluña', 'Toledo': 'Castilla-La Mancha',
    'Santiago de Compostela': 'Galicia','Sevilla': 'Andalucía','Vitoria/Gasteiz': 'País Vasco',
    'Vigo': 'Galicia','Viladecans': 'Cataluña','Fuenlabrada': 'Comunidad de Madrid',
    'Gandia': 'Comunidad Valenciana','Torremolinos': 'Andalucía','El Prat de Llobregat': 'Cataluña',
    'Valencia': 'Comunidad Valenciana','Valladolid': 'Castilla y León','Tomares': 'Andalucía',
    'Tarragona': 'Cataluña','Torrevieja': 'Comunidad Valenciana','Eivissa': 'Islas Baleares',
    'el Masnou': 'Cataluña','Torrelavega': 'Cantabria','Cuenca': 'Castilla-La Mancha',
    'Ferrol': 'Galicia','Guadalajara': 'Castilla-La Mancha','Huelva': 'Andalucía',
    'Igualada': 'Cataluña','Vilanova i la Geltrú': 'Cataluña','Granada': 'Andalucía',
    'Zamora': 'Castilla y León','Granollers': 'Cataluña','Torrejón de Ardoz': 'Comunidad de Madrid',
    'Gijón': 'Asturias','Santander': 'Cantabria','Córdoba': 'Andalucía','Talavera de la Reina': 'Castilla-La Mancha',
    'León': 'Castilla y León','Linares': 'Andalucía','Línea de la Concepción, La': 'Andalucía',
    'Lleida': 'Cataluña','Logroño': 'La Rioja',"El Port de Sagunt / Canet d'En Berenguer": 'Comunidad Valenciana',
    'Dos Hermanas': 'Andalucía','Irun': 'País Vasco','Madrid': 'Comunidad de Madrid',
    'Majadahonda': 'Comunidad de Madrid','Fuengirola': 'Andalucía','Valdemoro': 'Comunidad de Madrid',
    'Alcalá de Guadaíra': 'Andalucía','Coslada': 'Comunidad de Madrid','Oviedo': 'Asturias',
    'Algeciras': 'Andalucía','San Sebastián/Donostia': 'País Vasco','Sanlúcar de Barrameda': 'Andalucía',
    'Almería': 'Andalucía','Ávila': 'Castilla y León','Palma de Mallorca': 'Islas Baleares','Pamplona/Iruña': 'Navarra','Avilés': 'Asturias',
    'Pontevedra': 'Galicia','Pozuelo de Alarcón': 'Comunidad de Madrid','Alicante/Alacant': 'Comunidad Valenciana',
    'Palencia': 'Castilla y León','Zaragoza': 'Aragón','Girona': 'Cataluña','Sabadell': 'Cataluña',
    'Salamanca': 'Castilla y León','San Fernando': 'Andalucía','Manresa': 'Cataluña','Marbella': 'Andalucía','Parla': 'Comunidad de Madrid',
    'Ponferrada': 'Castilla y León', 'Melilla': 'Ciudad Autónoma de Melilla','A Coruña': 'Galicia','Albacete': 'Castilla-La Mancha',
    'Mérida': 'Extremadura','Murcia': 'Región de Murcia','Alcalá de Henares': 'Comunidad de Madrid',
    'Ourense': 'Galicia','Alcoi/Alcoy': 'Comunidad Valenciana','Sant Cugat del Vallès/Rubí': 'Cataluña',
    'Sant Boi de Llobregat': 'Cataluña',
    'Jaén': 'Andalucía',
    'Jerez de la Frontera': 'Andalucía',
    'Lorca': 'Región de Murcia',
    'Lugo': 'Galicia',
    'Benidorm': 'Comunidad Valenciana',
    'Bilbao': 'País Vasco',
    'Puerto de Santa María, El': 'Andalucía','Reus': 'Cataluña','Ciudad Real': 'Castilla-La Mancha',
    'Badajoz': 'Extremadura','Burgos': 'Castilla y León','Cáceres': 'Extremadura',
    'Barcelona': 'Cataluña','Málaga': 'Andalucía','Elche/Elx': 'Comunidad Valenciana',
    'Elda': 'Comunidad Valenciana','Mataró': 'Cataluña','Cádiz': 'Andalucía',
    'Cartagena': 'Región de Murcia','Castelldefels': 'Cataluña','Mollet del Vallès': 'Cataluña',
    'Castellón de la Plana/Castelló de la Plana': 'Comunidad Valenciana','Ceuta': 'Ciudad Autónoma de Ceuta',
    'Chiclana de la Frontera': 'Andalucía'
}

df["CCAA"] = df["City Or Territory"].map(mapa_ccaa)

In [ ]:
df["CCAA"].unique() # visualizamos los resultados

array(['Comunidad de Madrid', 'Cataluña', 'Castilla-La Mancha', 'Galicia',
       'Andalucía', 'Cantabria', 'Islas Baleares', 'Comunidad Valenciana',
       'Castilla y León', 'País Vasco', 'Asturias', 'Aragón', 'La Rioja',
       'Región de Murcia', 'Ciudad Autónoma de Melilla', 'Extremadura',
       'Navarra', 'Ciudad Autónoma de Ceuta', nan], dtype=object)

### Filtrado filas y columnas DataFrame Original

Filtramos las filas por:
* Ciudades que son de la `Urban Centres (grid)`
* Los que se atribuyen a las muertes `Health Indicator = Attributable deaths (AD)`.
* Edades: `>= 25 years of age"` y `>= 30 years of age`

Por columnas nos quedaremos con `CCAA`,`City Or Territory`, `Air Pollutant`, `Year`, `Air Pollution Average [ug/m3]`, `Air Pollution Population Weighted Average [ug/m3]`, `Affected  Population`, `Populated Area [km2]`.

In [ ]:
df = df[
    (df["City Boundary Specification (LAU/grid)"] == "Urban Centres (grid)") &
    (df["Description Of Age Group"].isin([">= 25 years of age", ">= 30 years of age"])) &
    (df["Health Indicator"] == "Attributable deaths (AD)")
]

In [ ]:
df = df.sort_values( # ordenamos el dataset
    by=["CCAA", "City Or Territory", "Year", "Air Pollutant"]
).reset_index(drop=True)

Vamos a seleccionar columnas de interés y eliminar duplicados.

In [ ]:
columnas = ['CCAA','City Or Territory','Air Pollutant','Year',
            'Air Pollution Average [ug/m3]',
            'Air Pollution Population Weighted Average [ug/m3]',
            'Affected  Population', 'Populated Area [km2]']

df = df[columnas].copy()

In [ ]:
df = df.drop_duplicates()

Comprobamos que todas las ciudades reúnen las mismas características de estudio y eliminamos aquellos registros que añadan información irrelevante por ser un dato duplicado.

In [ ]:
pd.set_option('display.max_rows', None)
df["City Or Territory"].value_counts() == 15


,count
City Or Territory,
Gijón,False
Irun,False
Eivissa,False
Alcalá de Guadaíra,True
Cádiz,True
Córdoba,True
Almería,True
Algeciras,True
Fuengirola,True


In [ ]:
indices_eliminar = [642, 639, 2555, 2560, 2440]
df = df.drop(index=indices_eliminar)

# **DATOS "LIMPIOS"**

El dataset tiene 1568 instancias y 8 variables de estudio.

In [ ]:
df.shape

(1563, 8)

In [ ]:
df.head(4)

,CCAA,City Or Territory,Air Pollutant,Year,Air Pollution Average [ug/m3],Air Pollution Population Weighted Average [ug/m3],Affected Population,Populated Area [km2]
0,Andalucía,Alcalá de Guadaíra,NO2,2018,17.9,18.9,47706,11.0
1,Andalucía,Alcalá de Guadaíra,PM2.5,2018,10.2,10.2,47706,11.0
2,Andalucía,Alcalá de Guadaíra,NO2,2019,16.0,17.0,48026,11.0
3,Andalucía,Alcalá de Guadaíra,PM2.5,2019,10.2,10.2,48026,11.0


In [ ]:
df[df["City Or Territory"] == "Eivissa"] # ejemplo de ciudad (todas tienen las mismas caracteríticas)


,CCAA,City Or Territory,Air Pollutant,Year,Air Pollution Average [ug/m3],Air Pollution Population Weighted Average [ug/m3],Affected Population,Populated Area [km2]
2418,Islas Baleares,Eivissa,NO2,2018,11.7,13.1,30730,8.8
2419,Islas Baleares,Eivissa,PM2.5,2018,7.8,7.9,30730,8.8
2420,Islas Baleares,Eivissa,NO2,2019,11.1,12.5,30936,8.8
2421,Islas Baleares,Eivissa,PM2.5,2019,9.0,9.0,30936,8.8
2422,Islas Baleares,Eivissa,NO2,2020,8.6,9.9,31215,8.8
2423,Islas Baleares,Eivissa,PM2.5,2020,8.3,8.4,31215,8.8
2424,Islas Baleares,Eivissa,NO2,2021,8.1,9.5,33711,8.8
2425,Islas Baleares,Eivissa,NO2,2021,8.1,9.5,31327,8.8
2427,Islas Baleares,Eivissa,PM2.5,2021,7.6,7.7,33711,8.8
2428,Islas Baleares,Eivissa,PM2.5,2021,7.6,7.7,31327,8.8
